In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
from fastai.vision import *
from fastai.metrics import accuracy
from fastai.metrics import error_rate

In [ ]:
# run constants
bs = 64
size = 224
stage1_lr = 3e-3
srage2_lr = slice(1e-6, 1e-4)
normalize = True
model = models.resnet50
#res34: bs64, size224
#res18: bs1024, size300

In [ ]:
untar_path = untar_data(URLs.PETS)
img_path = untar_path/'images'
label_path = untar_path/'annotations'
img_fnames = get_image_files(img_path)

np.random.seed(2)
pattern = r'/([^/]+)_\d+.jpg$'
data = ImageDataBunch.from_name_re(img_path, img_fnames, pattern, ds_tfms = get_transforms(), size=size, bs=bs)
if normalize: data = data.normalize(imagenet_stats)

In [ ]:
learn.destroy()
learn = None
gc.collect()

In [ ]:
learn = cnn_learner(data, model, metrics=error_rate)

In [ ]:
learn.lr_find()

In [ ]:
learn.recorder.plot()

In [ ]:
learn.unfreeze()

In [ ]:
learn.lr_find()

In [ ]:
learn.recorder.plot()

In [ ]:
learn.summary()

In [ ]:
learn.fit_one_cycle(4, max_lr = stage1_lr)

In [ ]:
learn.unfreeze()

In [ ]:
learn.fit_one_cycle(2, max_lr=srage2_lr)

In [ ]:
metric = learn.recorder.metrics[-1][0]

In [ ]:
learn.destroy()

## Grid search

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
from fastai.vision import *
from fastai.metrics import accuracy
from fastai.metrics import error_rate
import pandas as pd
gc.collect()

0

In [2]:
# run constants
g_bs = [16, 32, 64]
g_size = [100, 150, 200, 224]
stage1_lr = 3e-3
srage2_lr = slice(1e-6, 1e-4)
normalize = True
model = models.resnet50

In [3]:
df = pd.DataFrame(columns=['bs', 'size', 'model', 'normalize', 'error_rate'])

In [4]:
#df = df.append({'bs': bs, 'size': size, 'model':model.__name__, 'normalize':normalize, 'error_rate':0.08}, ignore_index=True)
#df

In [5]:
for size in g_size:
    for bs in g_bs:
        
        print('model: {0}, {1}'.format(bs, size))
        untar_path = untar_data(URLs.PETS)
        img_path = untar_path/'images'
        label_path = untar_path/'annotations'
        img_fnames = get_image_files(img_path)

        np.random.seed(2)
        pattern = r'/([^/]+)_\d+.jpg$'
        data = ImageDataBunch.from_name_re(img_path, img_fnames, pattern, ds_tfms = get_transforms(), size=size, bs=bs)
        if normalize: data = data.normalize(imagenet_stats)

        learn = cnn_learner(data, model, metrics=error_rate)
        learn.fit_one_cycle(4, max_lr = stage1_lr)
        learn.unfreeze()
        learn.fit_one_cycle(2, max_lr=srage2_lr)
        metric = learn.recorder.metrics[-1][0]
        learn.destroy()
        
        df = df.append({'bs': bs, 'size': size, 'model':model.__name__, 'normalize':normalize, 'error_rate':metric}, ignore_index=True)

model: 16, 100


epoch,train_loss,valid_loss,error_rate,time
0,1.630078,1.036856,0.292287,00:23
1,1.479348,1.127226,0.311231,00:20
2,0.981675,0.598948,0.182679,00:20
3,0.781827,0.519369,0.168471,00:20


epoch,train_loss,valid_loss,error_rate,time
0,0.655864,0.498993,0.167118,00:20
1,0.630916,0.484749,0.152909,00:21


this Learner object self-destroyed - it still exists, but no longer usable
model: 32, 100


epoch,train_loss,valid_loss,error_rate,time
0,1.446317,0.823946,0.244249,00:14
1,1.136686,0.802615,0.244249,00:13
2,0.801196,0.517328,0.174560,00:13
3,0.574043,0.474967,0.154939,00:13


...

epoch,train_loss,valid_loss,error_rate,time
0,0.199463,0.211152,0.071042,00:39
1,0.175177,0.204189,0.070365,00:39


this Learner object self-destroyed - it still exists, but no longer usable
model: 64, 224


epoch,train_loss,valid_loss,error_rate,time
0,0.835244,0.316451,0.094723,00:31
1,0.487711,0.330670,0.094723,00:30
2,0.308052,0.223235,0.064953,00:29
3,0.183956,0.195527,0.058863,00:29


epoch,train_loss,valid_loss,error_rate,time
0,0.176147,0.188356,0.054804,00:39
1,0.135193,0.186287,0.056834,00:37


this Learner object self-destroyed - it still exists, but no longer usable


In [6]:
df.to_pickle('lect1-res50.pickle')

In [ ]:
#learn.destroy()